In [1]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import face_recognition

In [1]:
from face_detection import FaceDetector
detector = FaceDetector()

RuntimeError: Unable to open shape_predictor_81_face_landmarks.dat

In [ ]:
def load_data(folder_path):
    image_paths = []
    face_shape_categories = ["Heart", "Oval", "Oblong", "Round", "Square"]
    labels = []
    features = []
    
    for category_index, category in enumerate(face_shape_categories):
        category_folder = os.path.join(folder_path, category)
        
        for image_name in os.listdir(category_folder):
            image_path = os.path.join(category_folder, image_name)
            dp_extarcted_features = extract_face_features(image_path)

            if dp_extarcted_features is not None:
                fn_extracted_features = detector.get_measuremet_values(image_path)
                image_paths.append(image_path)
                combined_f = np.concatenate((dp_extarcted_features, fn_extracted_features))
                features.append(combined_f)
                labels.append(category_index)

    return labels, features

def extract_face_features(image_path):
    img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_img)

    if len(face_locations) != 0:
        extracted_features = face_recognition.face_encodings(rgb_img, known_face_locations=face_locations)[0]
        return extracted_features


In [ ]:
def train():
        current_dir = os.getcwd()
        dataset_folder_training = os.path.join(current_dir, 'Dataset2\\training_set')

        train_labels, train_features = load_data(dataset_folder_training)

        X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)
        model = LGBMClassifier(n_estimators=100, random_state=42, reg_alpha=0.1, reg_lambda=0.1, max_depth=11)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        train_accuracy = accuracy_score(y_train, y_train_pred)
        val_accuracy = accuracy_score(y_val, y_val_pred)

        print("Training Accuracy:", train_accuracy)
        print("Validation Accuracy:", val_accuracy)

        dataset_folder_testing = os.path.join(current_dir, 'Dataset2\\testing_set')
        test_labels, test_features = load_data(dataset_folder_testing)

        y_test_pred = model.predict(test_features)
        test_accuracy = accuracy_score(test_labels, y_test_pred)
        print("Testing Accuracy:", test_accuracy)

        report = classification_report(y_val, y_val_pred)
        print(report)

        # Confusion Matrix
        confusion_matrix = confusion_matrix(y_val, y_val_pred)
        print(confusion_matrix)

        return model


In [ ]:
def save_model(model):
    with open('model.pkl', "wb") as file:
        pickle.dump(model, file)

def load_model():
    with open('model.pkl', "rb") as file:
       model = pickle.load(file)
